#### 한국어 코퍼스 전처리
- 데이터셋 : Korpora 에서 로딩
- 형태소분석기 설정
- 단어사전 생성

[1] 모듈로딩 및 데이터 준비 <hr>

In [33]:
# Module Loading
from Korpora import Korpora                             # Open Korean DataSet
from konlpy.tag import *                                # 형태소 분석기
import spacy                                            # 형태소 분석기

from torchtext.data.utils import get_tokenizer          # Pytorch 자연어처리 Module
from torchtext.vocab import *                           # Pytorch 자연어처리 단어사전 관련 Module
from torch.utils.data import Dataset, DataLoader        # Pytorch DataSet 관련 Module

import pandas as pd
import numpy as np


In [34]:
## 데이터 로딩
nsmc=Korpora.load("nsmc")


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/

[Korpora] Corpus `nsmc` is already installed at C:\Users\KDP-27\Korpora\nsmc\ratings_train.txt
[Korpora] Corpus `nsmc` is already installed at C:\Users\KD

In [35]:
nsmc.train[0], nsmc.test[0]

(LabeledSentence(text='아 더빙.. 진짜 짜증나네요 목소리', label=0),
 LabeledSentence(text='굳 ㅋ', label=1))

In [36]:
# Test 데이터셋을 DataFrame으로 로딩
nsmcDF=pd.DataFrame(nsmc.test)          # 데이터양이 많아서 test로 확인, 원래는 train데이터 변환해야함
nsmcDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    50000 non-null  object
 1   label   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


[2] 커스컴 데이터셋 정의 <hr>

In [44]:
### 단어 사전을 생성시 활용

class TextDataset(Dataset):
    def __init__(self, feature, label):
        self.feature = feature
        self.label=label
        self.lenght = feature.shape[0]

    def __len__(self):
        return self.lenght

    def __getitem__(self, index):
        return self.feature.iloc[index], self.label.iloc[index]

In [45]:
## NSMC의 데이터셋 인스턴스 생성
nsmcDS=TextDataset(nsmcDF['text'],nsmcDF['label'])

In [46]:
for f, l in nsmcDS:
    print(f,l)
    break       # 데이터 양 많아서 break 안하면 죽음

굳 ㅋ 1


[3] 텍스트 데이터 전처리<hr>
- 토큰화 / 정제 (불용어, 구두점, 오타, ......)
- 단어사전 생성
- 문장 ==> 수치화

In [47]:
# def preprosessing(text):      # 여기서 전처리 함수 많들어서 사용 또는 generateToken() 안에서 전처리 해서 사용 또는 TextDataset class에서 전처리
#     #불용어 제거
#     pass

In [48]:
#### 토커나이즈 생성
okt_tokenizer=get_tokenizer(Okt().morphs)         # morphs()는 함수 호출이라 morphs로 해야 함수 주소가 넘어감
spacy_tokenizer=get_tokenizer('spacy',language='ko_core_news_sm')

In [49]:
### 택스트 Dataset에서 처리할 데이터셋만 로딩해서 토큰화 진행하는 반복자 함수 (전체 다 넣으면 데이터 많아서 죽음)

def generateToken(dataset):
    for text, label in dataset:
        
        #tokens=okt_tokenizer(text)

        yield okt_tokenizer(text)


In [50]:
### 단어 사전 생성
UNK_TOKEN='<UNK>'
PAD_TOKEN='<PAD>'

nsmcVocab=build_vocab_from_iterator(iterator=generateToken(nsmcDS),
                                    specials=[UNK_TOKEN])     

nsmcVocab.set_default_index( nsmcVocab[UNK_TOKEN])    # UMK -> OOV 가능

In [51]:
nsmcVocab

Vocab()